In [1]:
import sys
import os
os.chdir("../")

In [8]:
import pandas as pd
import data
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None
#pd.options.display.max_colwidth = -1

In [3]:
train_df = data.train_df('small')

In [4]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1


In [5]:
train_df = train_df.sort_index()

In [27]:
clickout_rows = train_df[['user_id','session_id','action_type','impressions','prices']][train_df.action_type == 'clickout item']
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|')
# save the sorted prices for each clickout
clickout_rows['price_list'] = clickout_rows.prices.str.split('|').apply(lambda x: list(map(int,x)))
clickout_rows['sorted_price_list'] = clickout_rows.price_list.apply(lambda x: sorted(x))
clickout_rows = clickout_rows.drop('prices', axis=1)

reference_rows = train_df[['user_id','session_id','reference','action_type']][train_df.reference.str.isnumeric() & (train_df.action_type != 'clickout item')].drop('action_type',axis=1)
reference_rows['price_pos'] = -1

In [32]:
clickout_rows.head()

,user_id,session_id,action_type,impressions,impression_list,price_list,sorted_price_list
7,00RL8Z82B2Z1,aff3928535f48,clickout item,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,"[3400638, 1253714, 3367857, 5100540, 1088584, 666916, 54833, 2922310, 9711560, 109038, 666856, 10077318, 1431482, 129343, 6339822, 6806806, 1041528, 109013, 3909420, 55088, 3095758, 109018, 54885, 1257342, 2595006]","[95, 66, 501, 112, 95, 100, 101, 72, 82, 56, 56, 143, 70, 25, 71, 162, 73, 143, 188, 118, 77, 131, 143, 49, 165]","[25, 49, 56, 56, 66, 70, 71, 72, 73, 77, 82, 95, 95, 100, 101, 112, 118, 131, 143, 143, 143, 162, 165, 188, 501]"
9,00RL8Z82B2Z1,aff3928535f48,clickout item,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,"[55109, 129343, 54824, 2297972, 109014, 1257342, 1031578, 109018, 1332971, 666916, 54833, 54885, 2237222, 10077318, 1166793, 9132132, 1474297, 3909420, 6622154, 55091, 8444418, 54804, 2050977, 666936, 55088]","[162, 25, 150, 143, 101, 49, 118, 131, 18, 100, 101, 143, 51, 143, 123, 36, 66, 188, 124, 138, 94, 137, 180, 112, 118]","[18, 25, 36, 49, 51, 66, 94, 100, 101, 101, 112, 118, 118, 123, 124, 131, 137, 138, 143, 143, 143, 150, 162, 180, 188]"
21,02SRUT1NQYH1,3599a6f709eab,clickout item,2795374|5582964|1088390|2781070|1258068|1271962|3184892|148884|3528776|107183|5156744|107048|1907333|3370484|6003326|8436316|9025316|125181|3861490|131257|4415954|107162|3143352|6652864|8118684,"[2795374, 5582964, 1088390, 2781070, 1258068, 1271962, 3184892, 148884, 3528776, 107183, 5156744, 107048, 1907333, 3370484, 6003326, 8436316, 9025316, 125181, 3861490, 131257, 4415954, 107162, 3143352, 6652864, 8118684]","[64, 54, 36, 121, 76, 81, 92, 40, 73, 52, 98, 104, 56, 414, 67, 111, 21, 122, 55, 104, 56, 64, 40, 29, 44]","[21, 29, 36, 40, 40, 44, 52, 54, 55, 56, 56, 64, 64, 67, 73, 76, 81, 92, 98, 104, 104, 111, 121, 122, 414]"
26,03K8AXBL4BX2,ec139e10b9238,clickout item,12693|46363|81657|18448|47687|152913|18417|927627|4147572|819616|18380|47333|18425|1255377|4043482|1032816|18388|1216900|106471|106567|12667|84082|1152188|98737|47410,"[12693, 46363, 81657, 18448, 47687, 152913, 18417, 927627, 4147572, 819616, 18380, 47333, 18425, 1255377, 4043482, 1032816, 18388, 1216900, 106471, 106567, 12667, 84082, 1152188, 98737, 47410]","[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, 107, 86, 98, 83, 98, 103, 105, 99, 106, 99, 98, 96, 96, 105, 99]","[72, 75, 81, 83, 85, 86, 92, 96, 96, 98, 98, 98, 99, 99, 99, 100, 102, 103, 103, 104, 104, 105, 105, 106, 107]"
43,03P4VFKK12UO,325fafb5fa450,clickout item,1306936|56482|2842358|6881276|65685|63259|65399|1320460|360566|376756|885381|60769|2141484|375876|829246|58909|360571|1298518|63542|71926|376076|57803|376181|1117192|359326,"[1306936, 56482, 2842358, 6881276, 65685, 63259, 65399, 1320460, 360566, 376756, 885381, 60769, 2141484, 375876, 829246, 58909, 360571, 1298518, 63542, 71926, 376076, 57803, 376181, 1117192, 359326]","[178, 104, 110, 94, 57, 96, 46, 61, 48, 35, 50, 38, 59, 44, 57, 56, 55, 45, 146, 41, 44, 44, 39, 38, 126]","[35, 38, 38, 39, 41, 44, 44, 44, 45, 46, 48, 50, 55, 56, 57, 57, 59, 61, 94, 96, 104, 110, 126, 146, 178]"


In [29]:
reference_rows.head(30)

,user_id,session_id,reference,price_pos
1,00RL8Z82B2Z1,aff3928535f48,666856,-1
2,00RL8Z82B2Z1,aff3928535f48,109038,-1
3,00RL8Z82B2Z1,aff3928535f48,666856,-1
4,00RL8Z82B2Z1,aff3928535f48,109038,-1
5,00RL8Z82B2Z1,aff3928535f48,666856,-1
6,00RL8Z82B2Z1,aff3928535f48,109038,-1
10,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
11,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
12,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
13,02HGRBA06ODU,fa3a53156ae5c,3066244,-1


In [30]:
#reference_rows['ref_indice'] = -1
#reference_rows['lista_imp'] = ''
#reference_rows['prezzi_ordinati'] = 'a'
#reference_rows['prezzi'] = 'a'

j = 0
clickout_indices = clickout_rows.index.values
for idx,row in tqdm(reference_rows.iterrows()):
    # find the next clickout index
    if idx >= clickout_indices[-1]:
        break
    while idx > clickout_indices[j]:
        j += 1
    next_clickout = clickout_rows.loc[clickout_indices[j]]

    # check if row and next_clickout are in the same session
    if row.user_id == next_clickout.user_id and row.session_id == next_clickout.session_id:
        try:
            ref_idx = next_clickout.impression_list.index(row.reference)
            ref_price = int(next_clickout.price_list[ref_idx])
            reference_rows.at[idx,'price_pos'] = next_clickout.sorted_price_list.index(ref_price)
            #reference_rows.at[idx,'ref_indice'] = ref_idx
            #reference_rows.at[idx,'lista_imp'] = ','.join(next_clickout.impression_list)
            #reference_rows.at[idx, 'prezzi'] = str(next_clickout.price_list)
            #reference_rows.at[idx,'prezzi_ordinati'] = str(next_clickout.sorted_price_list)
        except:
            pass

In [31]:
reference_rows

,user_id,session_id,reference,price_pos
1,00RL8Z82B2Z1,aff3928535f48,666856,2
2,00RL8Z82B2Z1,aff3928535f48,109038,2
3,00RL8Z82B2Z1,aff3928535f48,666856,2
4,00RL8Z82B2Z1,aff3928535f48,109038,2
5,00RL8Z82B2Z1,aff3928535f48,666856,2
6,00RL8Z82B2Z1,aff3928535f48,109038,2
10,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
11,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
12,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
13,02HGRBA06ODU,fa3a53156ae5c,3066244,-1


In [ ]:
reference_rows.dtypes

In [5]:
from extract_features.reference_price_position_in_next_clickout import ReferencePricePositionInNextClickout
f = ReferencePricePositionInNextClickout()

In [10]:
f.join_to(train_df, one_hot=True)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_price_position_in_next_clickout feature read
reference_price_position_in_next_clickout onehot completed


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,pricepos0,pricepos1,pricepos2,pricepos3,pricepos4,pricepos5,pricepos6,pricepos7,pricepos8,pricepos9,pricepos10,pricepos11,pricepos12,pricepos13,pricepos14,pricepos15,pricepos16,pricepos17,pricepos18,pricepos19,pricepos20,pricepos21,pricepos22,pricepos23,pricepos24
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
